# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234801342288                   -0.50    7.0
  2   -7.250266221020       -1.81       -1.40    1.0
  3   -7.251310990553       -2.98       -2.13    4.0
  4   -7.251284702261   +   -4.58       -2.26    4.0
  5   -7.251328032755       -4.36       -2.65    3.0
  6   -7.251337717395       -5.01       -3.11    2.0
  7   -7.251338764079       -5.98       -3.96    1.0
  8   -7.251338790039       -7.59       -4.07    4.0
  9   -7.251338797303       -8.14       -4.47    2.0
 10   -7.251338798222       -9.04       -4.71    2.0
 11   -7.251338798593       -9.43       -5.06    2.0
 12   -7.251338798701       -9.97       -5.76    2.0
 13   -7.251338798703      -11.58       -5.94    3.0
 14   -7.251338798704      -11.96       -6.42    2.0
 15   -7.251338798705      -12.86       -7.00    3.0
 16   -7.251338798705      -14.27       -7.30    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.033219945703425686
[ Info: Arnoldi iteration step 2: normres = 0.6103801057807471
[ Info: Arnoldi iteration step 3: normres = 0.6069369922929916
[ Info: Arnoldi iteration step 4: normres = 0.6105288316819663
[ Info: Arnoldi iteration step 5: normres = 0.35808542673846805
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.92e-02, 6.90e-02, 2.88e-01, 1.75e-01, 8.61e-03)
[ Info: Arnoldi iteration step 6: normres = 0.48043766716118147
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.80e-02, 1.15e-01, 4.15e-01, 1.40e-01, 6.96e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0992888665441085
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.02e-03, 1.89e-02, 3.51e-02, 5.90e-02, 5.98e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11855896620869556
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.07e